<a href="https://colab.research.google.com/github/assafrbi/mini-project-Digital-Humanities/blob/main/dh_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from requests import get
from bs4 import BeautifulSoup
import re
from time import sleep
from random import randint
from warnings import warn
from tqdm import tqdm
from imdb import Cinemagoer
import pandas as pd

### Load Data

In [10]:
url = "https://www.imdb.com/search/title/?title_type=feature&countries=il&start=1&ref_=adv_nxt"
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

num_films_text = html_soup.find_all('div', class_ = 'desc')
num_films=re.search('of (\d.+) titles',str(num_films_text[0])).group(1)
num_films=int(num_films.replace(',', ''))
print(num_films)

1471


In [13]:
ia = Cinemagoer()
movie_name = []
year = []
time=[]
rating=[]
votes = []
description = []
director_s = []
imdb_id = []
for start in tqdm(range(1, 500, 50)):    
    # Make a get request    
    url = "https://www.imdb.com/search/title/?title_type=feature&countries=il&start="+str(start)+"&ref_=adv_nxt"
    page = get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})

    for store in movie_data:
      name = store.h3.a.text
      movie_name.append(name)
      
      year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text if store.h3.find('span', class_ = "lister-item-year text-muted unbold") else " "
      year.append(year_of_release)
          
      runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') else " "
      time.append(runtime)
          
      rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '') if store.find('div', class_ = "inline-block ratings-imdb-rating") else " "
      rating.append(rate)
          
      value = store.find_all('span', attrs = {'name': "nv"})
          
      vote = value[0].text if store.find_all('span', attrs = {'name': "nv"}) else " "
      votes.append(vote)
          
      describe = store.find_all('p', class_ = 'text-muted')
      description_ = describe[1].text.replace('\n', '') if len(describe) > 1 else ' '
      description.append(description_)
            
      ps = store.find_all('p')
      for p in ps:
          if 'Director'in p.text:
              director =p.find('a').text
      
      director_s.append(director)
      
      imdbID = store.img.get('data-tconst')
      imdb_id.append(imdbID[2:])

100%|██████████| 10/10 [00:13<00:00,  1.30s/it]


In [14]:
df = pd.DataFrame(
    {'id': imdb_id,
     'movie': movie_name,
     'year': year,
     'synopsis': description,
     'time': time,
     'rating': rating,
     'votes': votes,
     'director': director_s,
     })
df

,id,movie,year,synopsis,time,rating,votes,director
0,9100054,The Lost Daughter,(2021),A woman's beach vacation takes a dark turn whe...,121 min,6.7,"59,735",Maggie Gyllenhaal
1,3605418,Knock Knock,(I) (2015),A devoted father helps two stranded young wome...,99 min,4.9,"93,191",Eli Roth
2,0090859,Cobra,(1986),A tough-on-crime street cop must protect the o...,87 min,5.8,"72,664",George P. Cosmatos
3,3454424,Where Is Anne Frank,(2021),"The film follows the journey of Kitty, the ima...",99 min,6.3,665,Ari Folman
4,0090927,The Delta Force,(1986),"After a plane is hijacked by terrorists, The D...",125 min,5.6,"20,393",Menahem Golan
...,...,...,...,...,...,...,...,...
495,0830833,A Dangerous Dance,(2007),"Joseph, a young handsome unemployed from a sma...",90 min,2.4,85,Menahem Golan
496,0105416,Tel Aviv Stories,(1992),Three stories about three women: one is trying...,108 min,7.0,84,Ayelet Menahemi
497,0141591,Siege,(1969),Story of a war-widow pressured by her friends ...,100 min,7.5,98,Gilberto Tofano
498,8551348,On the Edge: Gesta 2,(2018),Adam falls victim to a sting and loses all his...,94 min,4.8,14,Kobi Machat


In [15]:
from google.colab import drive
drive.mount('/content/gdrive')
with open('/content/gdrive/My Drive/movies.txt', 'w') as f:
  f.write(df.to_csv())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Text Preprocessing


### Topic Modeling - Latent Dirichlet Allocation